In [ ]:
#recommended to use through google colab

#path file from Drive 
import os
from google.colab import drive
drive.mount('/content/drive')
#change directory to you drive and connect to folder varuna hackathon
root_path = "/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022" 

#saw file in directory
!ls "$root_path"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Grid_AOI	       sentinel-2-image   training_area
'Presentation Slide'   testing_area


# install


In [ ]:
!pip install rasterio
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# BOund

In [ ]:
import rasterio
import rasterio.mask
import geopandas as gpd

import matplotlib.pyplot as plt
from rasterio import plot

# path_2021 = f'{root_path}/sentinel-2-image/2021'
# path_2020 = f'{root_path}/sentinel-2-image/2020'

# def read_band(year_path, date, band_name):
#     date_path = f'{year_path}/{date}'
#     band_path = f'{date_path}/IMG_DATA/47PQS_{date}_{band_name}.jp2'
#     band = rasterio.open(band_path)
#     return band


In [ ]:
# train_shape = gpd.read_file(f'{root_path}/training_area/traindata.shp')
# test_shape = gpd.read_file(f'{root_path}/testing_area/testdata.shp')

# train_crs = train_shape.to_crs({'init': 'epsg:32647'})
# test_crs = test_shape.to_crs({'init': 'epsg:32647'})

In [ ]:
# date = '20210101'
# band2 = read_band(path_2021, date, 'B02')
# band3 = read_band(path_2021, date, 'B03')
# band4 = read_band(path_2021, date, 'B04')

In [ ]:
# aoi = read_band(path_2021, date, 'AOT')
# scl = read_band(path_2021, date, 'SCL')

In [ ]:
# aoi.transform

# extractdata


In [ ]:
#open jp2 and return array and scl label
def SCL_converter(jp2):
  ds = rasterio.open(jp2)
  dat_array=ds.read(1)
  dict_label={}

  for i in range(0,13):
    dict_label[i]=(dat_array == i).sum()

  return dat_array,dict_label

# def SCL_carray_conv(arr):
#   dict_label={}

#   for i in range(0,13):
#     dict_label[i]=(arr == i).sum()  


#   return dict_label

#function for scale coordination from geometry to location in array
def scale_x(x):

  res=(x-754360)*(1025/20500)

  return int(res)
def scale_y(y):

  res=(y-1668180)*(1025/20500)

  return int(res)

def filter_scl(arr):
  mem=[]
  for i in arr:
    if int(i)==4 or int(i)==5 or int(i)==6 or int(i)==7:
      mem.append(i)
    else:
      pass
  return mem

def mode_2(arr):
  mem=[]
  c=0
  set_=set(arr)
  for i in set_:
    c1=arr.count(i)
    if c1==c:
      mem.append(i)
      c=c1
    elif c1>c:
      mem=[i]
      c=c1
    else:
      pass
  return mem




# run Train

In [ ]:
#function for get scl only 4,5,6,7 
import geopandas as gpd
import pandas as pd
import numpy as np

#connect to testing area
df_train=gpd.read_file("/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/training_area/traindata.shp")
df_train["X_cen"]=df_train.centroid.x
df_train["Y_cen"]=df_train.centroid.y
df_train["conx"]=df_train["X_cen"].apply(scale_x)
df_train["cony"]=df_train["Y_cen"].apply(scale_y)
df_train

,years,crop_type,geometry,X_cen,Y_cen,conx,cony
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517
...,...,...,...,...,...,...,...
1312,2021,3,"POLYGON ((769004.617 1679027.544, 769130.199 1...",769189.041539,1.679080e+06,741,544
1313,2021,3,"POLYGON ((772624.509 1673384.254, 772770.291 1...",772697.815894,1.673360e+06,916,258
1314,2021,1,"POLYGON ((774333.243 1669788.192, 774337.269 1...",774379.730345,1.669879e+06,1000,84
1315,2021,3,"POLYGON ((768880.988 1687424.354, 768996.565 1...",768928.241627,1.687336e+06,728,957


In [ ]:
import os 
import pandas as pd


#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202101'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_1"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:17<00:00, 58.28it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202102'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_2"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 90.51it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202103'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_3"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:21<00:00, 47.11it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202104'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_4"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 86.70it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4


In [ ]:
import os 
import pandas as pd
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202105'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_5"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:12<00:00, 79.76it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202106'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_6"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 87.14it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202107'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_7"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 86.60it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202108'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_8"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 88.77it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5,4
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5,4


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202109'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_9"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:11<00:00, 87.56it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5,4,7
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5,4,6


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202110'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_10"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:12<00:00, 84.33it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5,4,7,4
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4,4,4,9
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5,4,6,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202111'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_11"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:09<00:00, 110.08it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5,4,7,4,4
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4,4,4,4,4
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4,4,4,9,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5,4,6,5,5


In [ ]:
import os 
import pandas as pd
#change directory to you drive and connect to folder varuna hackathon in sentinel-2-image years 2021
#root_folder="/content/drive/Shareddrives/ARV_HACKATHON/Varuna Hackathon 2022/sentinel-2-image/2021/"

list_of_date=os.listdir(root_folder)

# for i in list_of_date[:10]:
#   if i.startswith('202101'):
name_interset=[]
array_list_interest=[]
for i in list_of_date:
  if i.startswith('202112'):
  
    path_folder=root_folder+i+"/IMG_DATA/"
    list_of_scl=os.listdir(path_folder)
  #print(list_of_scl)

    for j in list_of_scl:
      if (".jp2" in j) and ("SCL" in j) and (".xml" not in j):
        dat_path=root_folder+i+"/IMG_DATA/"+j
        dar=SCL_converter(dat_path)

        #print(j)
        #print(dat_path)
        name_interset.append(dat_path)
        array_list_interest.append(dar[0])
        #display(plot.show(dar[0]))
        #display(pd.DataFrame.from_dict(dar[1],orient='index').T)

my_rows, my_cols = (1025, 1025)
my_array = [[0]*my_cols]*my_rows
my_array=np.array(my_array)
#print(my_array)



from tqdm import tqdm 

for j in tqdm(range(1025)):
  for k in range(1025):
    list_mode=[]
    for i in range(len(array_list_interest)):
      list_mode.append(array_list_interest[i][j][k])
    #print(list_mode)
    if filter_scl(list_mode)==[]:
      #print("here")
      pass
    else:
      list_mode=filter_scl(list_mode)


    my_array[j][k]=mode_2(list_mode)[0]

    


lis=[]
for i,j in zip(df_train.conx,df_train.cony):
  lis.append(my_array[i][j])


df_train["m_12"]=lis
df_train.head()


100%|██████████| 1025/1025 [00:10<00:00, 99.41it/s]


,years,crop_type,geometry,X_cen,Y_cen,conx,cony,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12
0,2021,1,"POLYGON ((774604.014 1671240.055, 774606.506 1...",774642.520254,1.671280e+06,1014,154,5,5,5,5,5,9,5,4,7,4,4,4
1,2021,1,"POLYGON ((773367.837 1670278.438, 773389.600 1...",773467.890415,1.670263e+06,955,104,5,5,5,4,4,4,4,4,4,4,4,5
2,2021,1,"POLYGON ((771648.671 1673990.960, 771707.027 1...",771755.834321,1.674121e+06,869,297,4,5,5,4,4,4,4,4,4,4,4,4
3,2021,3,"POLYGON ((771306.117 1674174.738, 771426.678 1...",771357.710386,1.674136e+06,849,297,4,4,4,5,4,4,4,4,4,9,4,4
4,2021,2,"POLYGON ((763750.339 1678588.825, 763879.691 1...",763844.138010,1.678522e+06,474,517,5,5,5,4,5,5,5,4,6,5,5,5


# result

In [ ]:
df_train_sent=df_train.filter(like="m_", axis=1)
df_train_sent

,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12
0,5,5,5,5,5,9,5,4,7,4,4,4
1,5,5,5,4,4,4,4,4,4,4,4,5
2,4,5,5,4,4,4,4,4,4,4,4,4
3,4,4,4,5,4,4,4,4,4,9,4,4
4,5,5,5,4,5,5,5,4,6,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1312,4,4,5,4,4,5,5,4,4,4,4,4
1313,5,5,5,5,5,7,5,5,6,4,4,4
1314,5,5,5,5,4,4,5,4,4,4,4,5
1315,5,5,5,4,4,4,5,4,4,9,4,5


In [ ]:
# #import file from STEP1
df_tm=pd.read_csv("/content/train_max_evi_ndwi_savi_b1.csv")
df_tm

In [ ]:
df_train_sent_join=pd.concat([df_tm,df_train_sent],axis=1)
df_train_sent_join=df_train_sent_join.drop(columns="years")
df_train_sent_join

In [ ]:
df_train_sent_join.to_csv("/content/train_set.csv",index=False)